<a href="https://colab.research.google.com/github/hhhhh0102/ner_project/blob/main/ner_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
!git clone https://github.com/hhhhh0102/ner_project.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import os
import sys
sys.path.append("/content/ner_project/ner_model")

In [6]:
import numpy as np
import tensorflow as tf
from Preprocess import preprocess
from make_model import make_model
from tokenizer import KoBertTokenizer

def ner_task(textpath):
    tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
    model = tf.keras.models.load_model("/content/drive/MyDrive/my_model.h5")
    my_model = make_model()
    my_model.model_input.set_dataset("/content/ner_project/ner_model/train_data")
    my_model.model_input.make_input()
    line_list = []

    f = open(textpath, mode='r', encoding='utf-8')
    while True:
        line = f.readline()
        if not line: break
        line_list.append(line)
    f.close()

    f = open('ner_result.txt', mode='wt', encoding='utf-8')
    for l in list(range(len(line_list))):

        tokenized_sentence = np.array([tokenizer.encode(line_list[l], max_length=88, pad_to_max_length=True)])
        tokenized_mask = np.array([[int(x != 1) for x in tokenized_sentence[0].tolist()]])
        ans = model.predict([tokenized_sentence, tokenized_mask])
        ans = np.argmax(ans, axis=2)

        tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence[0])
        new_tokens, new_labels = [], []
        for token, label_idx in zip(tokens, ans[0]):

            if (token.startswith("▁")):
                new_labels.append(my_model.model_input.pr.index_to_ner[label_idx])
                new_tokens.append(token[1:])
            elif (token == '[CLS]'):
                pass
            elif (token == '[SEP]'):
                pass
            elif (token == '[PAD]'):
                pass
            elif (token != '[CLS]' or token != '[SEP]'):
                new_tokens[-1] = new_tokens[-1] + token

        for ll in list(range(len(new_tokens))):
            f.write("{}\t{}\t{}\n".format(ll + 1, new_tokens[ll], new_labels[ll]))
        f.write("\n")

    f.close()

In [ ]:
ner_task("/content/ner_project/pre_trained_data.txt")